In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp
import math

In [3]:
pd.set_option('display.float_format', '{:.2f}'.format)
data = pd.read_excel('99543018.xlsx', sheet_name='2022')
data

,interview__key,date_q,hh_02,settlement,members,memnal,weight,fdpurch,fdcons,fdout,...,hous_29_0__3,hous_29_0__4,hous_29_0__5,hous_29_0__6,hous_29_0__7,hous_39,hous_44,soc_01,soc_05,hous_47
0,00-00-85-41,2022.04,1,1,1,1,294.30,52468.75,0.00,0.00,...,1,-999999999,-999999999,-999999999,-999999999,1,3,2,NaN,1.00
1,00-03-45-50,2022.08,10,2,4,4,40.75,36217.56,38990.90,0.00,...,0,0,0,1,0,1,3,2,NaN,1.00
2,00-09-00-97,2022.04,4,2,4,4,192.16,19879.46,8436.28,0.00,...,0,0,0,0,0,2,4,2,NaN,1.00
3,00-10-35-11,2022.09,4,1,3,3,109.82,84514.88,499.84,0.00,...,1,0,0,0,0,1,2,2,NaN,1.00
4,00-11-16-10,2022.03,4,1,6,6,109.82,64809.23,0.00,0.00,...,0,0,0,1,0,1,4,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5179,99-96-72-90,2022.06,1,1,3,3,317.29,86318.15,3559.20,0.00,...,1,0,0,0,0,1,3,2,NaN,1.00
5180,99-96-79-06,2022.10,11,2,5,5,99.80,100461.90,8804.39,6300.60,...,0,0,0,1,0,1,3,1,5.00,1.00
5181,99-97-55-91,2022.06,3,1,5,1,91.06,27248.26,4706.84,0.00,...,1,0,0,0,0,1,3,2,NaN,1.00
5182,99-98-03-60,2022.10,1,1,1,1,92.66,29330.36,0.00,1738.10,...,1,0,0,0,0,1,3,2,NaN,2.00


In [4]:
labels = pd.read_excel('99543018.xlsx', sheet_name='Labels')
labels

,Variable,Label,Unnamed: 2,Unnamed: 3,Unnamed: 4,Variable Values,Unnamed: 6,Unnamed: 7
0,interview__key,Special relation record number of hh,NaN,NaN,NaN,Value,NaN,Label
1,date_q,Date of Survey,NaN,NaN,NaN,hh_02,-999999999.00,missing
2,hh_02,Code of marz,NaN,NaN,NaN,NaN,1.00,YEREVAN
3,settlement,Type of the area,NaN,NaN,NaN,NaN,2.00,ARAGATSOT
4,members,Number of the members of the household,NaN,NaN,NaN,NaN,3.00,ARARAT
...,...,...,...,...,...,...,...,...
62,hous_39,39. What is the principal method of garbage di...,NaN,NaN,NaN,NaN,2.00,no
63,hous_44,44. Please evaluate your housing condition,NaN,NaN,NaN,NaN,3.00,dont know
64,soc_01,1. Is your family registered in the poverty ...,NaN,NaN,NaN,NaN,NaN,NaN
65,soc_05,5. How many years has your family received b...,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5184 entries, 0 to 5183
Data columns (total 67 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   interview__key    5184 non-null   object 
 1   date_q            5184 non-null   float64
 2   hh_02             5184 non-null   int64  
 3   settlement        5184 non-null   int64  
 4   members           5184 non-null   int64  
 5   memnal            5184 non-null   int64  
 6   weight            5184 non-null   float64
 7   fdpurch           5184 non-null   float64
 8   fdcons            5184 non-null   float64
 9   fdout             5184 non-null   float64
 10  Z1                5184 non-null   int64  
 11  nfdpurch          5184 non-null   float64
 12  nonfdgif          5184 non-null   float64
 13  durble            5184 non-null   float64
 14  expend            5184 non-null   float64
 15  monincome         5184 non-null   float64
 16  totincome         5184 non-null   float64


# Data Transformation

In [7]:
# date looks like 2022.04 in float, let's change it to datetime
data['date_q'] = data['date_q'].astype(str)

data['date_q'] = data['date_q'].str.replace('.', '-')

data['date_q'] = pd.to_datetime(data['date_q'], format='%Y-%m').dt.to_period('M')

data['date_q'].head()

0    2022-04
1    2022-08
2    2022-04
3    2022-09
4    2022-03
Name: date_q, dtype: period[M]

In [8]:
data['hh_02'].head()

0     1
1    10
2     4
3     4
4     4
Name: hh_02, dtype: int64

In [9]:
# we have region names for each code, so let's convert codes to region names and convert -999999999 to NaN

data.loc[:, 'settlement'] = data['settlement'].replace(-999999999, np.nan)

data['hh_02'] = data['hh_02'].astype('category')

category_labels = {
    1: 'YEREVAN',
    2: 'ARAGATSOT',
    3: 'ARARAT',
    4: 'ARMAVIR',
    5: 'GEGHARKUNIK',
    6: 'LORI',
    7: 'KOTAYK',
    8: 'SHIRAK',
    9: 'SYUNIK',
    10: 'VAYOTS DZOR',
    11: 'TAVUSH'
}
data['hh_02'] = data['hh_02'].cat.rename_categories(category_labels)

print(data['hh_02'].head()) 

0        YEREVAN
1    VAYOTS DZOR
2        ARMAVIR
3        ARMAVIR
4        ARMAVIR
Name: hh_02, dtype: category
Categories (11, object): ['YEREVAN', 'ARAGATSOT', 'ARARAT', 'ARMAVIR', ..., 'SHIRAK', 'SYUNIK', 'VAYOTS DZOR', 'TAVUSH']


In [10]:
# let's change the column name to region (hh_02 doesn't tell anything)
data = data.rename(columns={'hh_02': 'region'})
data.loc[:, 'settlement'] = data['settlement'].replace(-999999999, np.nan)

In [11]:
data['settlement'].head()

0    1
1    2
2    2
3    1
4    1
Name: settlement, dtype: int64

In [12]:
# settlement column should also be categorical

data['settlement'] = data['settlement'].replace(-999999999, np.nan)

data['settlement'] = data['settlement'].astype('category')

settlement_labels = {
    1: 'URBAN',
    2: 'RURAL'
}

data['settlement'] = data['settlement'].cat.rename_categories(settlement_labels)

data.settlement.head()

0    URBAN
1    RURAL
2    RURAL
3    URBAN
4    URBAN
Name: settlement, dtype: category
Categories (2, object): ['URBAN', 'RURAL']

In [13]:
unique_z1 = data['Z1'].unique()
print(unique_z1)

[0]


In [14]:
# let's drop the column as it contains only zeros, no meaningful information

data = data.drop(columns=['Z1'])

In [15]:
# assign readable names to columns

data.rename(columns={
    'fdpurch': 'food_purchase',
    'fdcons': 'food_consumption',
    'fdout': 'food_out_of_home',
    'Z1': 'small_food_purchase',
    'nfdpurch': 'non_food_purchase',
    'nonfdgif': 'non_food_gift',
    'durble': 'durable_goods',
    'expend': 'expenditure',
    'monincome': 'monetary_income',
    'totincome': 'total',
    'nonmoninc': 'non_monetary_income'
}, inplace=True)

In [16]:
# as income columns have too many missing values, let's add them all together to have one total income 

income_columns = [
    'y1_3amd.1.00', 'y1_3amd.2.00', 'y1_3amd.4.00', 'y1_3amd.6.00', 'y1_3amd.8.00',
    'y1_3amd.9.00', 'y1_3amd.10.00', 'y1_3amd.12.00', 'y1_3amd.13.00', 'y1_3amd.14.00',
    'y1_3amd.15.00', 'y1_3amd.17.00', 'y1_3amd.18.00', 'y1_3amd.19.00', 'y1_3amd.23.00',
    'y1_3amd.26.00', 'y1_3amd.28.00'
]

data['Total_Income'] = data[income_columns].sum(axis=1)

data['Total_Income'].head()

0    34000.00
1   113300.00
2   114500.00
3   600000.00
4   153000.00
Name: Total_Income, dtype: float64

In [17]:
data['Total_Income'].isnull().sum()

0

In [18]:
data = data.drop(columns=['y1_3amd.1.00', 'y1_3amd.2.00', 'y1_3amd.4.00', 'y1_3amd.6.00', 'y1_3amd.8.00',
    'y1_3amd.9.00', 'y1_3amd.10.00', 'y1_3amd.12.00', 'y1_3amd.13.00', 'y1_3amd.14.00',
    'y1_3amd.15.00', 'y1_3amd.17.00', 'y1_3amd.18.00', 'y1_3amd.19.00', 'y1_3amd.23.00',
    'y1_3amd.26.00', 'y1_3amd.28.00'])

In [19]:
#let's rename columns to have more readable names

data.rename(columns={
    'hhch_0_3_sum': 'members_0_3',
    'hhch_4_5_sum': 'members_4_5',
    'hhch_6_11_sum': 'members_6_11',
    'hhch_12_17_sum': 'members_12_17',
    'hheld_63_65_sum': 'members_63_65',
    'hheld_66_70_sum': 'members_66_70',
    'hheld_71_80_sum': 'members_71_80',
    'hheld_over81_sum': 'members_over_81',
    'hheld_over63_sum': 'members_over_63'
}, inplace=True)


# converting missing values to 0 in age groups, as each family told the number of people only of the age group they had family member, the other columns stayed empty

age_group_columns = ['members_0_3', 'members_4_5', 'members_6_11', 
                     'members_12_17', 'members_63_65', 'members_66_70', 
                     'members_71_80', 'members_over_81', 'members_over_63']

data[age_group_columns] = data[age_group_columns].fillna(0)


In [20]:
unique_sex = data['headsex'].unique()
unique_sex

array([2, 1])

In [21]:
data['headsex'] = data.headsex.replace({1: 'Male', 2: 'Female'})

data['headsex'] = data.headsex.astype('category')

data.headsex.head()

0    Female
1      Male
2    Female
3      Male
4      Male
Name: headsex, dtype: category
Categories (2, object): ['Female', 'Male']

In [22]:
unique_educ = data.headeduc.unique()
unique_educ

array([ 2.,  5.,  3.,  6.,  7., nan,  4.,  1.,  0.,  8.])

In [23]:
unique_marital = data.headmerstatus.unique()
unique_marital

array([3, 1, 4, 5, 2])

In [24]:
# let's label education levels

education_levels = {
    1.00: 'No primary, illiterate',
    2.00: 'No primary, literate',
    3.00: 'Primary',
    4.00: 'General (basic)',
    5.00: 'Secondary',
    6.00: 'Preliminary vocational',
    7.00: 'Middle vocational',
    8.00: 'Higher (Bachelor/Master)',
    9.00: 'Post-graduate',
    np.nan: 'Unknown'
}

data.headeduc = data.headeduc.replace(education_levels)
data.headeduc = data.headeduc.astype('category')

data.headeduc

0         No primary, literate
1                    Secondary
2         No primary, literate
3                    Secondary
4         No primary, literate
                 ...          
5179    Preliminary vocational
5180                   Primary
5181                   Primary
5182    Preliminary vocational
5183                   Primary
Name: headeduc, Length: 5184, dtype: category
Categories (10, object): [0.00, 'General (basic)', 'Higher (Bachelor/Master)', 'Middle vocational', ..., 'Preliminary vocational', 'Primary', 'Secondary', 'Unknown']

In [25]:
data.rename(columns={'headeduc': 'education_level'}, inplace=True)

In [26]:
# rename column for marital status and assign categorical values

data.rename(columns={'headmerstatus': 'marital_status'}, inplace=True)

marital_status_map = {
    1.00: 'Married',
    2.00: 'Never married',
    3.00: 'Widowed',
    4.00: 'Divorced/Separated',
    5.00: 'Cohabiting'
}

data['marital_status'] = data['marital_status'].replace(marital_status_map)

data['marital_status'] = data['marital_status'].astype('category')

data['marital_status']

0       Widowed
1       Married
2       Widowed
3       Married
4       Married
         ...   
5179    Married
5180    Widowed
5181    Widowed
5182    Widowed
5183    Married
Name: marital_status, Length: 5184, dtype: category
Categories (5, object): ['Cohabiting', 'Divorced/Separated', 'Married', 'Never married', 'Widowed']

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5184 entries, 0 to 5183
Data columns (total 50 columns):
 #   Column               Non-Null Count  Dtype    
---  ------               --------------  -----    
 0   interview__key       5184 non-null   object   
 1   date_q               5184 non-null   period[M]
 2   region               5184 non-null   category 
 3   settlement           5184 non-null   category 
 4   members              5184 non-null   int64    
 5   memnal               5184 non-null   int64    
 6   weight               5184 non-null   float64  
 7   food_purchase        5184 non-null   float64  
 8   food_consumption     5184 non-null   float64  
 9   food_out_of_home     5184 non-null   float64  
 10  non_food_purchase    5184 non-null   float64  
 11  non_food_gift        5184 non-null   float64  
 12  durable_goods        5184 non-null   float64  
 13  expenditure          5184 non-null   float64  
 14  monetary_income      5184 non-null   float64  
 15  tota

In [28]:
data.rename(columns={'aec_r': 'consumption_per_person'}, inplace=True)

In [29]:
data.rename(columns={'poverty': 'poverty_status'}, inplace=True)

poverty_status_map = {
    1.00: 'Non poor',
    2.00: 'Poor',
    3.00: 'Very poor'
}

data['poverty_status'] = data['poverty_status'].replace(poverty_status_map)

data['poverty_status'] = data['poverty_status'].astype('category')

data['poverty_status'].head()

0     Non poor
1     Non poor
2         Poor
3     Non poor
4    Very poor
Name: poverty_status, dtype: category
Categories (3, object): ['Non poor', 'Poor', 'Very poor']

In [30]:
data.rename(columns={'pweight': 'household_weight'}, inplace=True)

In [31]:
data.rename(columns={'ae_r': 'adult_equivalent_size'}, inplace=True)

In [32]:
data['hous_01'] = data['hous_01'].replace(-999999999, np.nan)

dwelling_type = {
    1.00: 'House',
    2.00: 'Apartment',
    3.00: 'Hostel',
    4.00: 'Railcar/container',
    5.00: 'Other'
}

data['hous_01'] = data['hous_01'].replace(dwelling_type)

data['hous_01'] = data['hous_01'].astype('category')

data = data.rename(columns={'hous_01': 'dwelling_type'})

data['dwelling_type']

0       Apartment
1           House
2           House
3       Apartment
4           House
          ...    
5179    Apartment
5180        House
5181    Apartment
5182    Apartment
5183        House
Name: dwelling_type, Length: 5184, dtype: category
Categories (5, object): ['Apartment', 'Hostel', 'House', 'Other', 'Railcar/container']

In [33]:
data['hous_13'] = data['hous_13'].replace(-999999999, np.nan)

water_source_map = {
    1.00: 'Centralized water supply',
    2.00: 'Spring water, well',
    3.00: 'Own system of water supply',
    4.00: 'River, lake',
    5.00: 'Delivered water',
    6.00: 'Bought water',
    7.00: 'Other'
}

data['hous_13'] = data['hous_13'].replace(water_source_map)

data['hous_13'] = data['hous_13'].astype('category')

data = data.rename(columns={'hous_13': 'water_source'})

data['water_source']

0         Centralized water supply
1         Centralized water supply
2         Centralized water supply
3         Centralized water supply
4         Centralized water supply
                   ...            
5179      Centralized water supply
5180    Own system of water supply
5181      Centralized water supply
5182      Centralized water supply
5183      Centralized water supply
Name: water_source, Length: 5184, dtype: category
Categories (6, object): ['Bought water', 'Centralized water supply', 'Delivered water', 'Other', 'Own system of water supply', 'Spring water, well']

In [34]:
# to handle missing values in water_tap_location, we wanted to check whether missing values depend on water source they chose

crosstab_result = pd.crosstab(data['water_source'], data['hous_16'].isnull(), margins=True)
print(crosstab_result)

hous_16                     False  True   All
water_source                                 
Bought water                    0     1     1
Centralized water supply     4932     0  4932
Delivered water                 0    16    16
Other                           0    23    23
Own system of water supply      0   166   166
Spring water, well              1    45    46
All                          4933   251  5184


False shows how many households provided their water tap location
True shows how many didn't provide a water tap location

So:
4933 households provided a valid water_tap_location and nearly all of them (4932) are using a centralized water supply. 

In [ ]:
data['hous_16'] = data['hous_16'].replace(-999999999, np.nan)

#replace nan values with a new unknown value

water_tap_location_map = {
    1.00: 'In the dwelling',
    2.00: 'In the yard',
    3.00: 'In the street',
    np.nan: 'Unknown / Not applicable'
}

data['hous_16'] = data['hous_16'].replace(water_tap_location_map)

data['hous_16'] = data['hous_16'].astype('category')

data = data.rename(columns={'hous_16': 'water_tap_location'})

data['water_tap_location']

In [ ]:
# create a new column 'heating_source' 
heating_source_columns = ['hous_29_0__1', 'hous_29_0__2', 'hous_29_0__3', 
                          'hous_29_0__4', 'hous_29_0__5', 'hous_29_0__6', 'hous_29_0__7']

# replace the missing value with 0 in this case
data[heating_source_columns] = data[heating_source_columns].replace(-999999999, 0)

heating_sources_map = {
    'hous_29_0__1': 'Central heating',
    'hous_29_0__2': 'Electricity',
    'hous_29_0__3': 'Natural gas',
    'hous_29_0__4': 'Liquefied gas',
    'hous_29_0__5': 'Oil and diesel',
    'hous_29_0__6': 'Wood',
    'hous_29_0__7': 'Coal'
}

data['heating_source'] = data[heating_source_columns].idxmax(axis=1).replace(heating_sources_map)

data.drop(columns=heating_source_columns, inplace=True)

data['heating_source'].head()


In [ ]:
data['hous_39'] = data['hous_39'].replace(-999999999, np.nan)

garbage_disposal = {
    1.00: 'Rubbish evacuation system',
    2.00: 'Collected by a dust-cart',
    3.00: 'Dumped by household members',
    4.00: 'Burned by household members',
    5.00: 'Buried by household members',
    6.00: 'Other'
}

data['hous_39'] = data['hous_39'].replace(garbage_disposal)

data['hous_39'] = data['hous_39'].astype('category')

data = data.rename(columns={'hous_39': 'garbage_disposal'})

data['garbage_disposal']

In [ ]:
data['hous_44'] = data['hous_44'].replace(-999999999, np.nan)

housing_condition_map = {
    1.00: 'Very good',
    2.00: 'Good',
    3.00: 'Satisfactory',
    4.00: 'Bad',
    5.00: 'Very bad'
}

data['hous_44'] = data['hous_44'].replace(housing_condition_map)

data['hous_44'] = data['hous_44'].astype('category')

data.rename(columns={'hous_44': 'housing_condition'}, inplace=True)

data = data.rename(columns={'hous_44': 'housing_condition'})

data['housing_condition'].head()

In [ ]:
data['soc_01'] = data['soc_01'].replace(-999999999, np.nan)


poverty_benefit_map = {
    1.00: 'YES',
    2.00: 'NO'
}

data['soc_01'] = data['soc_01'].replace(poverty_benefit_map)

data['soc_01'] = data['soc_01'].astype('category')

data = data.rename(columns={'soc_01': 'poverty_benefit'})

data['poverty_benefit']

In [ ]:
data['soc_05'] = data['soc_05'].replace(-999999999, 0)
data['soc_05'] = data['soc_05'].fillna(0)  # convert blanks to 0 as well

data['soc_05']

In [ ]:
satisfaction_map = {
    1.00: 'yes',
    2.00: 'no',
    3.00: 'don\'t know'
}
data['hous_47'] = data['hous_47'].replace(satisfaction_map)

data['hous_47'] = data['hous_47'].replace('', np.nan)
data['hous_47'] = data['hous_47'].fillna('don\'t know')

data['hous_47'] = data['hous_47'].astype('category')

data['hous_47'].head()

In [ ]:
data = data.rename(columns={'soc_05': 'years_of_benefit', 'hous_47': 'health_service_satisfaction'})

# Before and after data cleaning and transformation 

In [ ]:
data.to_excel("cleaned_data.xlsx", index=False)

In [ ]:
initial_data = pd.read_excel('99543018.xlsx', sheet_name='2022')

In [ ]:
initial_data.info()
data.info()

## Visualization of Expenditure

In [ ]:
#Box plot for expenditure
plt.figure(figsize=(14,6))
sns.boxplot(x=data['expenditure'], color = "slateblue")
plt.ticklabel_format(style='plain', axis='x')
plt.xticks(rotation=45, ha='right')
plt.xticks(np.arange(0,3150000, step = 100000))
plt.axvline(data['expenditure'].mean(), color = "pink")
plt.xlabel('Expenditure')
plt.show()

The boxplot reveals a right-skewed distribution with a concentration of data points on the lower end of the scale, and a substantial number of high outliers.
Most of the data falls between 0 and around 300,000, but a few extreme values go well beyond that, reaching over 3,000,000. These could represent a small group of individuals with much higher expenditures compared to the majority.
The mean is higher than the median, which supports the idea that outliers are significantly impacting the average.
This visualization helps identify that the majority of the data is clustered within a relatively smaller range, but there are extreme values that warrant further attention, possibly skewing overall statistics like the mean.

In [ ]:
#Histogram for expenditure
data['expenditure'].plot(kind = "hist", bins = 30, color = "orange", edgecolor = "black", figsize = (15,6))
plt.xticks(np.arange(0,3150000, step = 100000))
plt.show()

This histogram shows a right-skewed distribution of expenditure, where most values are concentrated between 0 and 300,000. 
The frequency of higher expenditure values drops sharply, indicating that only a small number of individuals have significantly higher expenditures.

In [ ]:
#Density plot for expenditure
data['expenditure'].plot(kind = "density", figsize = (14,6))
plt.xlim(0,3)
plt.axvline(data['expenditure'].mean(), color = "pink")
plt.axvline(data['expenditure'].median(), color = "red")
plt.xticks(np.arange(0,3150000, step = 100000))
plt.show()

This density plot shows a right-skewed distribution, where the majority of values are concentrated at the lower end, with a long tail extending towards higher values.
The red vertical lines likely represent key statistics, indicating that the mean is higher than the median due to the presence of a few high-value outliers.

## Visualization of Total Income

In [ ]:
data.Total_Income.describe()

In [ ]:
# Boxplot for Total Income

plt.figure(figsize=(14,6))
sns.boxplot(x=data['Total_Income'], color = "seagreen")
plt.ticklabel_format(style='plain', axis='x')
plt.xticks(rotation=45, ha='right')
plt.xticks(np.arange(0,5700000, step = 200000))
plt.axvline(data['Total_Income'].mean(), color = "yellow")
plt.xlabel('Total Income')
plt.show()

This box plot shows a highly skewed income distribution, with most of the data concentrated on the lower end (below 400,000) and a long tail of outliers extending to significantly higher income levels.
The presence of extreme outliers indicates a few individuals with much higher incomes, skewing the distribution.

In [ ]:
# Histogram for Total Income
data['Total_Income'].plot(kind = "hist", bins = 20, color = "seagreen", edgecolor = "black", figsize = (15,6))
plt.xticks(np.arange(0,5700000, step = 200000))
plt.show()

This histogram shows a right-skewed income distribution where the majority of individuals have incomes below 400,000, with a few individuals earning much higher amounts.
The long tail to the right indicates the presence of extreme outliers, similar to the previous boxplot.

In [ ]:
# Density plot for expenditure
data['Total_Income'].plot(kind = "density", figsize = (14,6), color = "seagreen")
plt.xlim(0,3)
plt.axvline(data['Total_Income'].mean(), color = "pink")
plt.axvline(data['Total_Income'].median(), color = "red")
plt.xticks(np.arange(0,5700000, step = 200000))
plt.show()

This density plot shows a highly right-skewed distribution of income, with the majority of values concentrated below 400,000.
The sharp decline in density as income increases reflects the presence of a few high-income outliers, pulling the tail of the distribution to the right.

Descriptive Statistics

In [ ]:
data.expenditure.describe()

# Descrptive Statistics

In [ ]:
# Descriptive statistics for expenditure variable

mean_exp = np.mean(data['expenditure'])
max_exp = np.max(data['expenditure'])
min_exp = np.min(data['expenditure'])
std_exp = np.std(data['expenditure'])
q1_exp = np.percentile(data['expenditure'],25)
median_exp = np.median(data['expenditure'])
q3_exp = np.percentile(data['expenditure'],75)
range_exp = np.max(data['expenditure']) - np.min(data['expenditure'])
var_exp = np.var(data['expenditure'])
skewness_exp = data['expenditure'].skew()
kurt_exp = data['expenditure'].kurt()

import scipy.stats as stats
from scipy.stats import skew
trimmed_mean_exp = stats.trim_mean(data.expenditure, proportiontocut=0.1)

expenditure_metrics = pd.DataFrame({
    'Metric': [
        'Mean', 'Max', 'Min', 'Standard Deviation', 
        'Q1', 'Median', 'Q3', 'Range', 
        'Variance', 'Skewness', 'Kurtosis', 'Trimmed Mean'
    ],
    'Value': [
        mean_exp, max_exp, min_exp, std_exp, 
        q1_exp, median_exp, q3_exp, range_exp, 
        var_exp, skewness_exp, kurt_exp, trimmed_mean_exp
    ]
})

expenditure_metrics

The average expenditure is 176,567.30 AMD, indicating a typical spending level, but the wide range from 8,815.48 AMD to 3,125,347.36 AMD highlights significant economic disparity among individuals.
The high standard deviation of 133,326.73 AMD and a skewness of 3.97 suggest that most people spend less than the mean, with a few individuals making disproportionately high expenditures.
Additionally, the trimmed mean of 157,414.38 AMD reveals a more moderate average when accounting for outliers, emphasizing the impact of extreme spending on the overall average.

In [ ]:
data.Total_Income.describe()

In [ ]:
# Descriptive statistics for Total Income variable 

mean_inc = np.mean(data['Total_Income'])
max_inc = np.max(data['Total_Income'])
min_inc = np.min(data['Total_Income'])
std_inc = np.std(data['Total_Income'])
q1_inc = np.percentile(data['Total_Income'],25)
median_inc = np.median(data['Total_Income'])
q3_inc = np.percentile(data['Total_Income'],75)
range_inc = np.max(data['Total_Income']) - np.min(data['Total_Income'])
var_inc = np.var(data['Total_Income'])
skewness_inc = data['Total_Income'].skew()
kurt_inc = data['Total_Income'].kurt()

import scipy.stats as stats
from scipy.stats import skew
trimmed_mean_inc = stats.trim_mean(data.Total_Income, proportiontocut=0.1)

income_metrics = pd.DataFrame({
    'Metric': [
        'Mean', 'Max', 'Min', 'Standard Deviation', 
        'Q1', 'Median', 'Q3', 'Range', 
        'Variance', 'Skewness', 'Kurtosis', 'Trimmed Mean'
    ],
    'Value': [
        mean_inc, max_inc, min_inc, std_inc, 
        q1_inc, median_inc, q3_inc, range_inc,
        var_inc, skewness_inc, kurt_inc, trimmed_mean_inc
    ]
})

income_metrics

The average total income is 226,915.02 AMD, but the extremely high maximum of 5,628,000.00 AMD and the presence of a minimum income of 0.00 AMD indicate substantial income inequality within the dataset.
A standard deviation of 230,370.14 AMD and a skewness of 8.37 further highlight that most individuals earn significantly less than the average, with a small number earning disproportionately high amounts.
The trimmed mean of 196,323.23 AMD suggests a more representative average when accounting for outliers, reinforcing the notion that extreme incomes heavily influence the overall average.

### Comparing Expenditure and Income results 

The analysis reveals significant disparities in both expenditure and income among individuals, with average expenditures at 176,567.30 AMD and total incomes averaging 226,915.02 AMD, yet both metrics are skewed by extreme values that indicate a small number of individuals have disproportionately high expenditures and incomes. The high standard deviations in both categories, alongside the presence of outliers, suggest the need for targeted economic policies to address the inequalities evident in spending and income distribution within the population.

In [ ]:
# Determining outliers for Total Income variable 

Q1 = data['expenditure'].quantile(0.25)
Q3 = data['expenditure'].quantile(0.75)

IQR = Q3 - Q1

lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

print(lower_limit)
print(upper_limit)

Outliers are between upper and lower limit, as lower limit = -116385.96875 and the minimum value = 0, that means we don't have outliers from the left which also we can notice from the boxplot. 
Our maximum_limit = 431876.30125, so we can consider outliers for our data number after it.

In [ ]:
data['income_category'] = pd.cut(data['Total_Income'], 
                                 bins = [-np.inf, 100000, 200000, 300000, 400000, np.inf], 
                                 labels = ['Up to 100K AMD', '100K-200K AMD', '200K-300K AMD', '300K-400K AMD', '400K AMD and higher'], 
                                 ordered=True)

In [ ]:
pd.crosstab(data['region'], 
           data['income_category'], 
           values = data['expenditure'],
           aggfunc="mean").round(2)

Yerevan, being the capital city, shows relatively high expenditures in each income category. The highest income group (400K AMD and higher) spends the most, with a mean expenditure of 324,688.64 AMD.
Yerevan's expenditures also increase significantly with income, suggesting that residents may have higher living costs or more consumption opportunities compared to rural areas.

Vayots Dzor shows higher expenditures in most income categories, particularly in the 400K AMD and higher group, where it ranks among the top with 308,520.38 AMD.
Kotayk and Gegharkunik show lower expenditures in the higher income brackets, possibly indicating either lower living costs or reduced consumption patterns in those regions.
Shirak shows relatively moderate expenditures across all income groups, with a slight upward progression. However, it's still lower in some categories compared to more urbanized areas.
Lori stands out for its steep rise in expenditure in higher income brackets, especially in the 400K AMD and higher category, where the expenditure climbs to 304,868.80 AMD.
Ararat has somewhat less variation across income categories compared to regions like Yerevan, with expenditures increasing but not as dramatically.

### One Sample t-test to check the null hypothesis that the average household expenditures in Armenia are 200000 AMD

In [ ]:
stats.ttest_1samp(a=data['expenditure'], 
                  popmean=200000)

T-statistic = -12.65
A large absolute value of the t-statistic suggests a significant difference between the sample mean and the hypothesized mean.

P-value = 3.61e-36
This is an extremely small p-value which indicates that such a difference is highly unlikely to occur by random chance alone.

Since the p-value is much smaller than any common significance level, we reject the null hypothesis that the average household expenditure in Armenia is 200,000 AMD. So, we have strong evidence to conclude that the average household expenditure in Armenia is not 200,000 AMD and it's smaller than the sample mean.

In [ ]:
sigma = data['expenditure'].std()/math.sqrt(len(data['expenditure']))    # Sample stdev/sample size

stats.t.interval(0.95,                      # Confidence level
                 df = len(data['expenditure']) - 1,                 # Degrees of freedom
                 loc = data['expenditure'].mean(),   # Sample mean
                 scale= sigma)              # Standard dev estimate

In [ ]:
data['expenditure'].mean().round(1)

### Independent Sample t-test to check the equality of average expenditures in different types of settlements

In [ ]:
np.unique(data["settlement"])

 Interpret the t-test results:
T-statistic: -0.2218
The t-statistic represents the difference in the means of urban and rural expenditures in terms of standard error units. A small t-statistic (close to zero) indicates that the means of the two groups are not significantly different.
P-value: 0.8245
The p-value represents the probability of observing this result (or something more extreme) if the null hypothesis is true.
In this case, the p-value of 0.8245 is much higher than common significance levels (e.g., 0.05 or 0.01). This means we do not have enough evidence to reject the null hypothesis.
Degrees of Freedom (df): 3909.8
This represents the degrees of freedom for the test, which depends on the sample sizes and variances of the groups.

In [ ]:
stats.ttest_ind(
    a=data.loc[data["settlement"] == 'URBAN', "expenditure"],
    b=data.loc[data["settlement"] == 'RURAL', "expenditure"],
    equal_var=False
)

The independent sample t-test did not find significant differences in expenditures between urban and rural households in Armenia at the 5% significance level.

### Paired Sample t-test to check average expenditures and income  equality

In [ ]:
stats.ttest_rel(data['expenditure'], data['Total_Income'])

Since the p-value is far below 0.05, we can confidently reject the null hypothesis, concluding that the average expenditures and income for households in Armenia are not equal.

### "Levels of Expenditures"

In [ ]:
data['expenditure'].describe()

In [ ]:
data['expenditure_level'] = pd.cut(
    data['expenditure'], 
    bins=[-np.inf, 50000, 150000, 250000, 500000, np.inf], 
    labels=['Very Low: up to 50000', 'Low: 50000 to 150000', 'Medium: 150000 to 250000', 'High: 250000 to 500000', 'Very High: above 500000']
)

In [ ]:
counts = data['expenditure_level'].value_counts(normalize=True)

In [ ]:
# Plot as pie chart
counts.plot(kind='pie', autopct='%1.1f%%', figsize=(5, 8), startangle=140,
                        colors=['lightcoral', 'lightskyblue', 'lightgreen', 'khaki', 'plum'])
plt.title("Expenditure Levels Distribution")
plt.ylabel('')  
plt.show()

### Chi-square test to check the independence of "Levels of Expenditures" from the "Marital Status"

In [ ]:
ct = pd.crosstab(data['expenditure_level'],data['marital_status'], normalize = True).mul(100).round(2)
ct

In [ ]:
stats.chi2_contingency(ct)

We fail to reject the null hypothesis. This implies that there is no significant association between "Levels of Expenditures" and "Marital Status."

### ANOVA to check the equality of means of Expenditures in different groups of the "Marital Status". 

Null Hypothesis (H₀): The mean expenditures are the same across all marital status groups.

Alternative Hypothesis (H₁): At least one marital status group has a different mean expenditure.

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.api as sm

model = ols('expenditure ~ marital_status', data=data).fit()
anova_table = sm.stats.anova_lm(model, type=1)

anova_table

### Additional analysis

In [ ]:
# ANOVA to compare mean expenditures across different education levels.
model = ols('expenditure ~ education_level', data=data).fit()
anova_table2 = sm.stats.anova_lm(model, type=2)
anova_table2

In [ ]:
# chi-square test on a crosstab between poverty_status and expenditure_level.
ct = pd.crosstab(data['poverty_status'], data['expenditure_level'], normalize = True).mul(100).round(2)
ct

In [ ]:
stats.chi2_contingency(ct)

In [ ]:
# dependency between income_category and expenditure_level
ct = pd.crosstab(data['income_category'], data['expenditure_level'])
ct

In [ ]:
# check whether if number of family members increases, the expenditure decreases

plt.scatter(data['members'], data['expenditure'], alpha=0.5)
plt.title("Expenditure based on Household size")
plt.xlabel("Household Size (Members)")
plt.ylabel("Expenditure")

plt.ticklabel_format(style='plain', axis='y')

plt.show()

In [ ]:
anova_model3 = ols('expenditure ~ education_level', data=data).fit()
anova_table3 = sm.stats.anova_lm(anova_model3, type=2)

anova_table3

In [ ]:
plt.figure(figsize=(12, 7))
sns.countplot(x='education_level', hue='expenditure_level', data=data)
plt.title("Distribution of Expenditure Levels by Education Level")
plt.xlabel("Education Level")
plt.ylabel("Count")
plt.xticks(rotation=45, ha='right')
plt.legend(title="Expenditure Level")
plt.show()